In [1]:
import pandas as pd
import datetime
import numpy as np
import json
import geocoder
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))

In [2]:
store_request=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/Store_Level_Zip_Selection/JT_20181105/Free Food Test - Candidate Stores 10.17.18.xlsx",
                            dtype=str,skiprows=2)
store_request=store_request[['Store#','DMA','Reg','Dist','Address','City','ST','Zip','DC','Group/Test Name']]

#
store_list_sftp=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20181101-134628-331.txt",
                              sep="|",dtype=str)
store_list_sftp=store_list_sftp[['location_id','longitude_meas','latitude_meas']].rename(columns={"location_id":"Store#"})
store_list_sftp['latitude_meas']=store_list_sftp['latitude_meas'].astype(float)
store_list_sftp['longitude_meas']=store_list_sftp['longitude_meas'].astype(float)

#
store_request=pd.merge(store_request,store_list_sftp,on="Store#",how="left")

In [3]:
store_level_P_S_T=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_P_S_T=store_level_P_S_T[store_level_P_S_T['location_id'].isin(store_request['Store#'])]
store_level_P_S_T=store_level_P_S_T[['location_id','revenue_flag','zip']].rename(columns={"location_id":"Store#"})
store_level_P_S_T=store_level_P_S_T[store_level_P_S_T['revenue_flag'].isin(["P","S"])]

In [4]:
store_request_in=store_request[store_request['Store#'].isin(store_level_P_S_T['Store#'])].reset_index()
store_request_out=store_request[~store_request['Store#'].isin(store_level_P_S_T['Store#'])].reset_index()
del store_request_in['index']
del store_request_out['index']

store_request_in=pd.merge(store_request_in,store_level_P_S_T,on="Store#",how="left")
store_request_in["distance"]=np.nan
for i in range(len(store_request_in)):
    store_lat=store_request_in['latitude_meas'][i]
    store_lng=store_request_in['longitude_meas'][i]  
    store_center=[store_lat,store_lng]
    zip_cd=store_request_in['zip'][i]  
    if zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
    else:
        dist=np.nan
    store_request_in['distance'][i]=dist
store_request_in=store_request_in[(store_request_in['revenue_flag']=="P") | (store_request_in['distance']<=20)]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
output_new_stores=pd.DataFrame()
for i in range(len(store_request_out)):
    store_num=store_request_out['Store#'][i]
    store_lat=store_request_out['latitude_meas'][i]
    store_lng=store_request_out['longitude_meas'][i]
    store_center=[store_lat,store_lng]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"Store#":store_num,"revenue_flag":"10_mile_zips","zip":zip_cd,"distance":dist},index=[i])
            output_new_stores=output_new_stores.append(df)
output_new_stores=output_new_stores[['Store#','revenue_flag','zip','distance']]
store_request_out=pd.merge(store_request_out,output_new_stores,on="Store#",how="left")

In [6]:
store_request_both=store_request_in.append(store_request_out)
store_request_both_to_merge=store_request_both[['Store#','revenue_flag','zip','distance']].rename(columns={"zip":"P_S_zips"})

output=pd.merge(store_request,store_request_both_to_merge,on="Store#",how="left")

df_unique_zips=output[['P_S_zips']].drop_duplicates()

In [8]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/Store_Level_Zip_Selection/JT_20181105/BL_Store level P S zips for 56 Free food test_JL_"+str(datetime.datetime.now().date())+".xlsx",
                     engine="xlsxwriter")
output.to_excel(writer,"P_S_zips_by_Store",index=False)
df_unique_zips.to_excel(writer,"unique_P_S_zips",index=False)
writer.save()